# 背景：在v1p8在验证集上的预测结果进行数据分析

## 导包、设置根目录

In [1]:
import pandas as pd
import numpy as np
import os
import nltk
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.word2vec import Word2Vec
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import random
import pickle
import multiprocessing
import re
import pickle
from matplotlib import pyplot as plt

from collections import Counter
from itertools import chain
from datetime import datetime

# 更改工作目录为当前项目根目录
import sys
import os
os.chdir(os.path.dirname(os.path.dirname(sys.path[0])))
print(os.getcwd())

/Users/jincan02/Projects/Log-diagnosis


## B榜最新数据的数据分析

之前的数据没变，新增测试集数据：

新增日志数据preliminary_sel_log_dataset_b.csv

新增报错数据preliminary_submit_dataset_b.csv

In [2]:
# 读取sel日志数据
sel_log_df = pd.read_csv('./pre_contest/dataset_B/preliminary_sel_log_dataset.csv').drop_duplicates()
# 读取训练标签数据：有重复数据！
train_label1 = pd.read_csv('./pre_contest/dataset_B/preliminary_train_label_dataset.csv')
train_label2 = pd.read_csv('./pre_contest/dataset_B/preliminary_train_label_dataset_s.csv')

# 
additional_sel_log_dataset = pd.read_csv('./pre_contest/dataset_B/additional_sel_log_dataset.csv')

#
preliminary_sel_log_dataset_a = pd.read_csv('./pre_contest/dataset_B/preliminary_sel_log_dataset_a.csv')

# 
preliminary_submit_dataset_a = pd.read_csv('./pre_contest/dataset_B/preliminary_submit_dataset_a.csv')

# 
preliminary_sel_log_dataset_b = pd.read_csv('./pre_contest/dataset_B/preliminary_sel_log_dataset_b.csv')

# 
preliminary_submit_dataset_b = pd.read_csv('./pre_contest/dataset_B/preliminary_submit_dataset_b.csv')

In [10]:
preliminary_submit_dataset_b

,sn,fault_time
0,0015fe530ad4,2020-05-01 23:48:17
1,00380f1435b0,2020-07-28 07:51:13
2,0045a71d0221,2020-07-02 06:33:54
3,004d5a7954e7,2020-08-24 08:27:55
4,004d5a7954e7,2020-08-24 09:42:45
...,...,...
3025,ff92904d80f2,2020-08-28 16:48:54
3026,ff9f7a9c5c7e,2020-04-14 08:13:55
3027,ffc2f37539a9,2020-02-21 22:52:54
3028,fff12c95cc99,2020-05-20 10:27:43


## 采用按照时间最近邻+截止时间间隔的方式匹配日志和标签

In [60]:
# 读取sel日志数据
sel_log_df = pd.read_csv('./pre_contest/dataset_B/preliminary_sel_log_dataset.csv').drop_duplicates()
# 读取训练标签数据：有重复数据！
train_label1 = pd.read_csv('./pre_contest/dataset_B/preliminary_train_label_dataset.csv')
train_label2 = pd.read_csv('./pre_contest/dataset_B/preliminary_train_label_dataset_s.csv')
train_label_df = pd.concat([train_label1,train_label2],axis=0).drop_duplicates()


# 合并日志和标签
sel_log_df['label'] = ''
train_label_df['time'] = train_label_df['fault_time']
train_label_df['msg'] = ''
train_label_df['server_model'] = train_label_df['sn'].map(dict(zip(sel_log_df['sn'],sel_log_df['server_model'])))
train_label_df = train_label_df[['sn', 'time', 'msg', 'server_model', 'label']]

log_label_df = pd.concat([sel_log_df,train_label_df], axis = 0).sort_values(by = 'time')
log_label_df['fault_time'] = ''
log_label_df = log_label_df[['sn', 'fault_time', 'msg', 'time', 'server_model', 'label']]

In [61]:
log_label_df

,sn,fault_time,msg,time,server_model,label
330876,SERVER_10657,,Memory CPU1C0_DIMM_Stat | Correctable ECC | A...,2019-12-27 23:38:05,SM54,
328692,SERVER_10657,,Memory CPU1C0_DIMM_Stat | Correctable ECC | A...,2019-12-27 23:38:19,SM54,
329772,SERVER_10657,,Memory CPU1C0_DIMM_Stat | Correctable ECC | A...,2019-12-27 23:38:33,SM54,
329200,SERVER_10657,,Memory CPU1C0_DIMM_Stat | Correctable ECC | A...,2019-12-27 23:38:46,SM54,
330791,SERVER_10657,,Memory CPU1C0_DIMM_Stat | Correctable ECC | A...,2019-12-27 23:39:00,SM54,
...,...,...,...,...,...,...
267210,SERVER_20339,,Management Subsys Health System_Health | Sens...,2020-11-25 23:10:19,SM35,
291022,SERVER_20339,,Management Subsystem Health System_Health | S...,2020-11-25 23:10:19,SM35,
241259,SERVER_20339,,System Boot Initiated BIOS_Boot_Up | Initiate...,2020-11-25 23:11:23,SM35,
261330,SERVER_20339,,System Boot Initiated BIOS_Boot_Up | Initiate...,2020-11-25 23:21:06,SM35,


In [115]:
# sn分组后，按照最近邻+时间间隔划分日志数据
def divideLogByNearestTime(log_label_df: pd.DataFrame):
    log_correspond_label_df = pd.DataFrame(columns=['sn', 'fault_time', 'msg', 'time', 'server_model', 'label'])
    no_label_log_list = []
    cutoff = 10 * 3600

    for sn, log in log_label_df.groupby('sn'):
        if len(log[log['label'] != '']) == 0:
            no_label_log_list.append(log)
        elif len(log[log['label'] != '']) == 1:
            msg_df = log[log['label'] == '']
            msg_df['label'] = log[log['label'] != '']['label'].iloc[0]
            msg_df['fault_time'] = log[log['label'] != '']['time'].iloc[0]
            log_correspond_label_df = pd.concat([log_correspond_label_df, msg_df])
        else:
            lable_df = log[log['label'] != '']
            msg_df = log[log['label'] == '']
            for msg_item in msg_df.iterrows():
                previous_delta_time = 1000 * 24 * 3600
                for lable_item in lable_df.iterrows():
                    now_delta_time = abs(datetime.strptime(lable_item[1]['time'],'%Y-%m-%d %H:%M:%S'
                        ) - datetime.strptime(msg_item[1]['time'],'%Y-%m-%d %H:%M:%S'))
                    if now_delta_time.days * 24 * 3600 + now_delta_time.seconds < previous_delta_time:
                        previous_delta_time = now_delta_time.days * 24 * 3600 + now_delta_time.seconds
                        final_lable = lable_item[1]
                        if previous_delta_time < cutoff:
                            msg_item[1]['fault_time'] = lable_item[1]['time']
                            msg_item[1]['label'] = lable_item[1]['label']
            log_correspond_label_df = pd.concat([log_correspond_label_df, msg_df]) 
    log_correspond_label_df = log_correspond_label_df[log_correspond_label_df['label'] != '']
    return log_correspond_label_df, no_label_log_list


# sn分组后，本次报错和上次报错之间的日志匹配到本次报错
def divideLogByFaultTime(log_label_df: pd.DataFrame):
    log_correspond_label_df = pd.DataFrame(columns=['sn', 'fault_time', 'msg', 'time', 'server_model', 'label'])
    no_label_log_list = []
    log_label_df =  log_label_df.reset_index(drop = True)
    
    for sn, log in log_label_df.groupby('sn'):
        if len(log[log['label'] != '']) == 0:
            no_label_log_list.append(log)
        elif len(log[log['label'] != '']) == 1:
            msg_df = log[log['label'] == '']
            msg_df['label'] = log[log['label'] != '']['label'].iloc[0]
            msg_df['fault_time'] = log[log['label'] != '']['time'].iloc[0]
            log_correspond_label_df = pd.concat([log_correspond_label_df, msg_df])
        else:
            # 使用index的顺序取数时，要注意
            cutoff_index = [-1] + log.loc[log['label'] != ''].index.tolist() + [log.index.tolist()[-1]+1]
            for kth in range(len(cutoff_index)-1):
                temp_log = log.loc[(log.index <= cutoff_index[kth+1]) & (log.index > cutoff_index[kth])]
                if len(temp_log) > 0:
                    if len(temp_log[temp_log['label'] != '']) == 0:
                        no_label_log_list.append(temp_log)
                    else:
                        msg_df = temp_log[temp_log['label'] == '']
                        msg_df['label'] = temp_log[temp_log['label'] != '']['label'].iloc[0]
                        msg_df['fault_time'] = temp_log[temp_log['label'] != '']['time'].iloc[0]
                        log_correspond_label_df = pd.concat([log_correspond_label_df, msg_df])
    return log_correspond_label_df, no_label_log_list


In [113]:
NearestTime_log_correspond_label_df, NearestTime_no_label_log_list = divideLogByNearestTime(log_label_df)
# FaultTime_log_correspond_label_df, FaultTime_no_label_log_list = divideLogByFaultTime(log_label_df)

/var/folders/bh/sv71wm991pb8sm02bw8rmrfc0000gp/T/ipykernel_29647/2668321568.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msg_df['label'] = log[log['label'] != '']['label'].iloc[0]
/var/folders/bh/sv71wm991pb8sm02bw8rmrfc0000gp/T/ipykernel_29647/2668321568.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msg_df['fault_time'] = log[log['label'] != '']['time'].iloc[0]


In [97]:
NearestTime_log_correspond_label_df

,sn,fault_time,msg,time,server_model,label,msg_hour,msg_minute
193524,SERVER_10001,2020-05-01 10:04:00,Processor CPU0_Status | IERR | Asserted,2020-05-01 08:54:43,SM57,1,8,54
193526,SERVER_10001,2020-05-01 10:04:00,Processor CPU1_Status | IERR | Asserted,2020-05-01 08:54:43,SM57,1,8,54
193529,SERVER_10001,2020-05-01 10:04:00,Management Subsys Health System_Health | Sens...,2020-05-01 08:55:03,SM57,1,8,55
193572,SERVER_10001,2020-05-01 10:04:00,Processor CPU1_Status | IERR | Deasserted,2020-05-01 08:59:48,SM57,1,8,59
193573,SERVER_10001,2020-05-01 10:04:00,Processor CPU0_Status | IERR | Deasserted,2020-05-01 08:59:48,SM57,1,8,59
...,...,...,...,...,...,...,...,...
442548,SERVER_9999,2020-10-13 02:57:00,Memory CPU1C0_DIMM_Stat | Uncorrectable ECC |...,2020-10-13 02:56:48,SM57,2,2,56
442549,SERVER_9999,2020-10-13 02:57:00,Memory CPU1C0_DIMM_Stat | Uncorrectable ECC |...,2020-10-13 02:56:56,SM57,2,2,56
442550,SERVER_9999,2020-10-13 02:57:00,Processor CPU1_Status | Configuration Error |...,2020-10-13 02:56:57,SM57,2,2,56
442552,SERVER_9999,2020-10-13 02:57:00,Memory CPU1C0_DIMM_Stat | Uncorrectable ECC |...,2020-10-13 02:57:03,SM57,2,2,57


In [98]:
FaultTime_log_correspond_label_df

,sn,fault_time,msg,time,server_model,label
193524,SERVER_10001,2020-05-01 10:04:00,Processor CPU0_Status | IERR | Asserted,2020-05-01 08:54:43,SM57,1
193526,SERVER_10001,2020-05-01 10:04:00,Processor CPU1_Status | IERR | Asserted,2020-05-01 08:54:43,SM57,1
193529,SERVER_10001,2020-05-01 10:04:00,Management Subsys Health System_Health | Sens...,2020-05-01 08:55:03,SM57,1
193572,SERVER_10001,2020-05-01 10:04:00,Processor CPU1_Status | IERR | Deasserted,2020-05-01 08:59:48,SM57,1
193573,SERVER_10001,2020-05-01 10:04:00,Processor CPU0_Status | IERR | Deasserted,2020-05-01 08:59:48,SM57,1
...,...,...,...,...,...,...
442548,SERVER_9999,2020-10-13 02:57:00,Memory CPU1C0_DIMM_Stat | Uncorrectable ECC |...,2020-10-13 02:56:48,SM57,2
442549,SERVER_9999,2020-10-13 02:57:00,Memory CPU1C0_DIMM_Stat | Uncorrectable ECC |...,2020-10-13 02:56:56,SM57,2
442550,SERVER_9999,2020-10-13 02:57:00,Processor CPU1_Status | Configuration Error |...,2020-10-13 02:56:57,SM57,2
442552,SERVER_9999,2020-10-13 02:57:00,Memory CPU1C0_DIMM_Stat | Uncorrectable ECC |...,2020-10-13 02:57:03,SM57,2


In [86]:
len(NearestTime_no_label_log_list)

0

In [90]:
len(FaultTime_no_label_log_list)

,sn,fault_time,msg,time,server_model,label
359740,SERVER_10616,,Memory #0xe2 | Correctable ECC | Asserted,2020-08-15 22:09:26,SM52,
359741,SERVER_10616,,Memory #0xe2 | Correctable ECC | Asserted,2020-08-15 22:09:52,SM52,


In [56]:
FaultTime_no_label_log_list[0]

,sn,fault_time,msg,time,server_model,label
251920,SERVER_10293,,Button Button_Pressed | Power Button pressed ...,2020-05-15 13:49:50,SM35,
241345,SERVER_10293,,Button Button_Pressed | Power Button pressed ...,2020-05-15 13:49:56,SM35,
216199,SERVER_10293,,Power Supply PS1_Status | Power Supply AC los...,2020-05-15 13:50:27,SM35,
232565,SERVER_10293,,"Power Supply PS2_Status | AC out-of-range, bu...",2020-05-15 13:50:28,SM35,
253473,SERVER_10293,,Microcontroller/Coprocessor BMC_Boot_Up | Dev...,2020-05-15 14:04:43,SM35,
218721,SERVER_10293,,Microcontroller BMC_Boot_Up | Device Enabled ...,2020-05-15 14:04:43,SM35,
238262,SERVER_10293,,Processor CPU0_Status | Presence detected | A...,2020-05-15 14:04:44,SM35,
264650,SERVER_10293,,Processor CPU1_Status | Presence detected | A...,2020-05-15 14:04:45,SM35,
216534,SERVER_10293,,System ACPI Power State ACPI_PWR_Status | S4/...,2020-05-15 14:04:45,SM35,
270907,SERVER_10293,,Power Supply PS1_Status | Presence detected |...,2020-05-15 14:05:13,SM35,


In [64]:
FaultTime_no_label_log_list[0]

,sn,fault_time,msg,time,server_model,label
251920,SERVER_10293,,Button Button_Pressed | Power Button pressed ...,2020-05-15 13:49:50,SM35,
241345,SERVER_10293,,Button Button_Pressed | Power Button pressed ...,2020-05-15 13:49:56,SM35,
216199,SERVER_10293,,Power Supply PS1_Status | Power Supply AC los...,2020-05-15 13:50:27,SM35,
232565,SERVER_10293,,"Power Supply PS2_Status | AC out-of-range, bu...",2020-05-15 13:50:28,SM35,
253473,SERVER_10293,,Microcontroller/Coprocessor BMC_Boot_Up | Dev...,2020-05-15 14:04:43,SM35,
218721,SERVER_10293,,Microcontroller BMC_Boot_Up | Device Enabled ...,2020-05-15 14:04:43,SM35,
238262,SERVER_10293,,Processor CPU0_Status | Presence detected | A...,2020-05-15 14:04:44,SM35,
264650,SERVER_10293,,Processor CPU1_Status | Presence detected | A...,2020-05-15 14:04:45,SM35,
216534,SERVER_10293,,System ACPI Power State ACPI_PWR_Status | S4/...,2020-05-15 14:04:45,SM35,
270907,SERVER_10293,,Power Supply PS1_Status | Presence detected |...,2020-05-15 14:05:13,SM35,


In [57]:
FaultTime_log_correspond_label_df[FaultTime_log_correspond_label_df['sn'] == 'SERVER_10293']

,sn,fault_time,msg,time,server_model,label,lable


In [58]:
log_label_df[log_label_df['sn'] == 'SERVER_10293']

,sn,fault_time,msg,time,server_model,label
251920,SERVER_10293,,Button Button_Pressed | Power Button pressed ...,2020-05-15 13:49:50,SM35,
241345,SERVER_10293,,Button Button_Pressed | Power Button pressed ...,2020-05-15 13:49:56,SM35,
288671,SERVER_10293,,System ACPI Power State ACPI_PWR_Status | S4/...,2020-05-15 13:49:58,SM35,
216199,SERVER_10293,,Power Supply PS1_Status | Power Supply AC los...,2020-05-15 13:50:27,SM35,
284166,SERVER_10293,,"Power Supply PS1_Status | AC out-of-range, bu...",2020-05-15 13:50:28,SM35,
287889,SERVER_10293,,Power Supply PS2_Status | Power Supply AC los...,2020-05-15 13:50:28,SM35,
232565,SERVER_10293,,"Power Supply PS2_Status | AC out-of-range, bu...",2020-05-15 13:50:28,SM35,
253473,SERVER_10293,,Microcontroller/Coprocessor BMC_Boot_Up | Dev...,2020-05-15 14:04:43,SM35,
218721,SERVER_10293,,Microcontroller BMC_Boot_Up | Device Enabled ...,2020-05-15 14:04:43,SM35,
238262,SERVER_10293,,Processor CPU0_Status | Presence detected | A...,2020-05-15 14:04:44,SM35,


## 添加时间和日志数的统计特征

In [116]:
for a,b in NearestTime_log_correspond_label_df.groupby(['sn', 'fault_time']):
    if len(b['label'].unique())>1:
        break

In [117]:
b

,sn,fault_time,msg,time,server_model,label
442535,SERVER_9999,2020-10-13 02:57:00,Processor CPU1_Status | Configuration Error |...,2020-10-13 02:56:11,SM57,2
442536,SERVER_9999,2020-10-13 02:57:00,Memory CPU1C0_DIMM_Stat | Uncorrectable ECC |...,2020-10-13 02:56:11,SM57,2
442539,SERVER_9999,2020-10-13 02:57:00,Processor CPU1_Status | Configuration Error |...,2020-10-13 02:56:18,SM57,2
442540,SERVER_9999,2020-10-13 02:57:00,Memory CPU1C0_DIMM_Stat | Uncorrectable ECC |...,2020-10-13 02:56:18,SM57,2
442541,SERVER_9999,2020-10-13 02:57:00,Processor CPU1_Status | Configuration Error |...,2020-10-13 02:56:26,SM57,2
442542,SERVER_9999,2020-10-13 02:57:00,Memory CPU1C0_DIMM_Stat | Uncorrectable ECC |...,2020-10-13 02:56:26,SM57,2
442543,SERVER_9999,2020-10-13 02:57:00,Memory CPU1C0_DIMM_Stat | Uncorrectable ECC |...,2020-10-13 02:56:33,SM57,2
442544,SERVER_9999,2020-10-13 02:57:00,Processor CPU1_Status | Configuration Error |...,2020-10-13 02:56:33,SM57,2
442545,SERVER_9999,2020-10-13 02:57:00,Memory CPU1C0_DIMM_Stat | Uncorrectable ECC |...,2020-10-13 02:56:41,SM57,2
442546,SERVER_9999,2020-10-13 02:57:00,Processor CPU1_Status | Configuration Error |...,2020-10-13 02:56:41,SM57,2


In [151]:
# 使用最近邻时间进行划分，划分后每个sn+fault_time，标签均唯一
use_log_label_df = NearestTime_log_correspond_label_df

# 使用报错时间截断进行划分，划分后每个sn+fault_time，标签均唯一
# use_log_label_df = FaultTime_log_correspond_label_df


use_log_label_df['msg_hour'] = use_log_label_df['time'].apply(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S").hour)
use_log_label_df['msg_minute'] = use_log_label_df['time'].apply(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S").minute)
use_log_label_df['fault_hour'] = use_log_label_df['fault_time'].apply(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S").hour)
use_log_label_df['fault_minute'] = use_log_label_df['fault_time'].apply(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S").minute)

# 最近一次日志时间距报错时间间隔，单位秒
nearest_msg_fault_time_delta_list = []

fault_minute_list = []

# 日志不去重时长度1,2,3,4日志数量统计
all_msg_1_cnt_list=[]
all_msg_2_cnt_list=[]
all_msg_3_cnt_list=[]
all_msg_4_cnt_list=[]

msg_1_cnt_list=[]
msg_2_cnt_list=[]
msg_3_cnt_list=[]
msg_4_cnt_list=[]
msg_hour_max_list=[]
msg_hour_min_list=[]
msg_hour_avg_list=[]
msg_hour_median_list=[]
msg_hour_mode_list=[]
msg_minute_max_list=[]
msg_minute_min_list=[]
msg_minute_avg_list=[]
msg_minute_median_list=[]
msg_minute_mode_list=[]


sn_list=[]
day_list=[]
server_model_list=[]
msg_log_list=[]
msg_cnt_list=[]
fault_hour_list=[]
label_list=[]
fault_time_list=[]
for msg_log_df in use_log_label_df.groupby(['sn','fault_time']):
    msg_log_str = ''
    all_msg_1_cnt = 0
    all_msg_2_cnt = 0
    all_msg_3_cnt = 0
    all_msg_4_cnt = 0
    msg_1_cnt = 0
    msg_2_cnt = 0
    msg_3_cnt = 0
    msg_4_cnt = 0
    for info in msg_log_df[1]['msg']:
    if info == info:
        if len(info.split('|')) == 1:
            all_msg_1_cnt += 1
        elif len(info.split('|')) == 2:
            all_msg_2_cnt += 1
        elif len(info.split('|')) == 3:
            all_msg_3_cnt += 1
        else:
            all_msg_4_cnt += 1
    for info in msg_log_df[1]['msg'].drop_duplicates():
        if info == info:
            msg_log_str=msg_log_str+info.lower()+'.'
            if len(info.split('|')) == 1:
                msg_1_cnt += 1
            elif len(info.split('|')) == 2:
                msg_2_cnt += 1
            elif len(info.split('|')) == 3:
                msg_3_cnt += 1
            else:
                msg_4_cnt += 1
    nearest_msg_fault_time_delta = abs(datetime.strptime(msg_log_df[1].iloc[-1]['time'],'%Y-%m-%d %H:%M:%S'
                    ) - datetime.strptime(msg_log_df[0][1],'%Y-%m-%d %H:%M:%S'))
    nearest_msg_fault_time_delta = nearest_msg_fault_time_delta.days * 24 * 3600 + nearest_msg_fault_time_delta.seconds
    sm=int(msg_log_df[1].iloc[0]['server_model'][2:])
    label=msg_log_df[1].iloc[0]['label']
    
    
    sn_list.append(msg_log_df[0][0])
    fault_time_list.append(msg_log_df[0][1])
    
    nearest_msg_fault_time_delta_list.append(nearest_msg_fault_time_delta)
    server_model_list.append(sm)
    msg_log_list.append(msg_log_str)
    msg_cnt_list.append(len(msg_log_df[1]))
    label_list.append(label)
    
    fault_hour_list.append(msg_log_df[1].iloc[0]['fault_hour'])
    fault_minute_list.append(msg_log_df[1].iloc[0]['fault_minute'])
    
    all_msg_1_cnt_list.append(all_msg_1_cnt)
    all_msg_2_cnt_list.append(all_msg_2_cnt)
    all_msg_3_cnt_list.append(all_msg_3_cnt)
    all_msg_4_cnt_list.append(all_msg_4_cnt)    
    
    msg_1_cnt_list.append(msg_1_cnt)
    msg_2_cnt_list.append(msg_2_cnt)
    msg_3_cnt_list.append(msg_3_cnt)
    msg_4_cnt_list.append(msg_4_cnt)
    
    msg_hour_max_list.append(msg_log_df[1]['msg_hour'].max())
    msg_hour_min_list.append(msg_log_df[1]['msg_hour'].min())
    msg_hour_avg_list.append(msg_log_df[1]['msg_hour'].mean())
    msg_hour_median_list.append(msg_log_df[1]['msg_hour'].median())
    msg_hour_mode_list.append(msg_log_df[1]['msg_hour'].mode()[0])
    
    msg_minute_max_list.append(msg_log_df[1]['msg_minute'].max())
    msg_minute_min_list.append(msg_log_df[1]['msg_minute'].min())
    msg_minute_avg_list.append(msg_log_df[1]['msg_minute'].mean())
    msg_minute_median_list.append(msg_log_df[1]['msg_minute'].median())
    msg_minute_mode_list.append(msg_log_df[1]['msg_minute'].mode()[0])
    
msg_log_label_df=pd.DataFrame(
    {
    'sn':sn_list,
    'fault_time':fault_time_list,
    'server_model':server_model_list,
    'msg_cnt':msg_cnt_list,
    'fault_hour':fault_hour_list,
    'fault_minute':fault_minute_list,
    'nearest_msg_fault_time_delta':nearest_msg_fault_time_delta_list,
    'all_msg_1_cnt':all_msg_1_cnt_list,
    'all_msg_2_cnt':all_msg_2_cnt_list,
    'all_msg_3_cnt':all_msg_3_cnt_list,
    'all_msg_4_cnt':all_msg_4_cnt_list,
    'msg_1_cnt':msg_1_cnt_list,
    'msg_2_cnt':msg_2_cnt_list,
    'msg_3_cnt':msg_3_cnt_list,
    'msg_4_cnt':msg_4_cnt_list,
    'msg_hour_max':msg_hour_max_list,
    'msg_hour_min':msg_hour_min_list,
    'msg_hour_avg':msg_hour_avg_list,
    'msg_hour_median':msg_hour_median_list,
    'msg_hour_mode':msg_hour_mode_list,
    'msg_minute_max':msg_minute_max_list,
    'msg_minute_min':msg_minute_min_list,
    'msg_minute_avg':msg_minute_avg_list,
    'msg_minute_median':msg_minute_median_list,
    'msg_minute_mode':msg_minute_mode_list,
    'msg_log':msg_log_list,
    'label':label_list
    })
msg_log_label_df.to_csv('./pre_contest/v1p8/msg_log_label_df.csv',sep=',',index=None)

In [152]:
msg_log_label_df

,sn,fault_time,server_model,msg_cnt,fault_hour,fault_minute,msg_1_cnt,msg_2_cnt,msg_3_cnt,msg_4_cnt,...,msg_hour_avg,msg_hour_median,msg_hour_mode,msg_minute_max,msg_minute_min,msg_minute_avg,msg_minute_median,msg_minute_mode,msg_log,label
0,SERVER_10001,2020-05-01 10:04:00,57,9,10,4,0,0,9,0,...,8.333333,8.0,8,59,0,38.333333,54.0,59,processor cpu0_status | ierr | asserted. proc...,1
1,SERVER_10003,2020-03-28 09:48:00,57,117,9,48,0,0,1,0,...,9.000000,9.0,9,48,45,46.444444,46.0,46,memory cpu1d0_dimm_stat | correctable ecc | a...,2
2,SERVER_10008,2020-02-25 16:12:00,53,5,16,12,0,0,3,0,...,15.000000,15.0,15,52,51,51.600000,52.0,52,processor cpu0_status | configuration error |...,1
3,SERVER_10008,2020-03-11 18:04:00,53,4,18,4,0,0,4,0,...,12.250000,16.0,16,52,46,47.750000,46.5,46,system boot initiated bios_boot_up | initiate...,2
4,SERVER_10009,2020-05-08 16:37:00,53,4,16,37,0,0,4,0,...,16.000000,16.0,16,7,7,7.000000,7.0,7,drive slot hdd_l_14_status | drive fault | as...,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15891,SERVER_9991,2020-08-04 22:49:00,56,2,22,49,0,0,1,0,...,20.500000,20.5,19,52,46,49.000000,49.0,46,memory cpu0a0_dimm_stat | correctable ecc | a...,2
15892,SERVER_9991,2020-10-07 18:42:00,56,1,18,42,0,0,1,0,...,18.000000,18.0,18,14,14,14.000000,14.0,14,memory cpu1a0_dimm_stat | correctable ecc | a...,2
15893,SERVER_9993,2020-05-14 23:50:00,57,2,23,50,0,0,1,0,...,23.000000,23.0,23,48,43,45.500000,45.5,43,memory cpu1f0_dimm_stat | correctable ecc | a...,2
15894,SERVER_9998,2020-05-29 11:25:00,57,2,11,25,0,0,2,0,...,11.000000,11.0,11,19,4,11.500000,11.5,4,memory cpu1e1_dimm_stat | correctable ecc | a...,2


## 读取日志和标签数据

In [153]:
# msg_log_label_df = pd.read_csv('./pre_contest/v1p7/msg_log_label_df.csv',sep=',')
msg_log_list=list(msg_log_label_df['msg_log'])
label_list=list(msg_log_label_df['label'])
msg_log_label_df

,sn,fault_time,server_model,msg_cnt,fault_hour,fault_minute,msg_1_cnt,msg_2_cnt,msg_3_cnt,msg_4_cnt,...,msg_hour_avg,msg_hour_median,msg_hour_mode,msg_minute_max,msg_minute_min,msg_minute_avg,msg_minute_median,msg_minute_mode,msg_log,label
0,SERVER_10001,2020-05-01 10:04:00,57,9,10,4,0,0,9,0,...,8.333333,8.0,8,59,0,38.333333,54.0,59,processor cpu0_status | ierr | asserted. proc...,1
1,SERVER_10003,2020-03-28 09:48:00,57,117,9,48,0,0,1,0,...,9.000000,9.0,9,48,45,46.444444,46.0,46,memory cpu1d0_dimm_stat | correctable ecc | a...,2
2,SERVER_10008,2020-02-25 16:12:00,53,5,16,12,0,0,3,0,...,15.000000,15.0,15,52,51,51.600000,52.0,52,processor cpu0_status | configuration error |...,1
3,SERVER_10008,2020-03-11 18:04:00,53,4,18,4,0,0,4,0,...,12.250000,16.0,16,52,46,47.750000,46.5,46,system boot initiated bios_boot_up | initiate...,2
4,SERVER_10009,2020-05-08 16:37:00,53,4,16,37,0,0,4,0,...,16.000000,16.0,16,7,7,7.000000,7.0,7,drive slot hdd_l_14_status | drive fault | as...,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15891,SERVER_9991,2020-08-04 22:49:00,56,2,22,49,0,0,1,0,...,20.500000,20.5,19,52,46,49.000000,49.0,46,memory cpu0a0_dimm_stat | correctable ecc | a...,2
15892,SERVER_9991,2020-10-07 18:42:00,56,1,18,42,0,0,1,0,...,18.000000,18.0,18,14,14,14.000000,14.0,14,memory cpu1a0_dimm_stat | correctable ecc | a...,2
15893,SERVER_9993,2020-05-14 23:50:00,57,2,23,50,0,0,1,0,...,23.000000,23.0,23,48,43,45.500000,45.5,43,memory cpu1f0_dimm_stat | correctable ecc | a...,2
15894,SERVER_9998,2020-05-29 11:25:00,57,2,11,25,0,0,2,0,...,11.000000,11.0,11,19,4,11.500000,11.5,4,memory cpu1e1_dimm_stat | correctable ecc | a...,2


## 读取v1_baseline用的词和v1p1的新词

In [154]:
v1_word_list=list(pd.read_csv('pre_contest/v1p2/word_frequency_df.txt',sep='\t')['word'])
v1p1_word_list=list(pd.read_csv('pre_contest/v1p2/tags_incomplete.txt',sep='\t',names=['word'])['word'])
v1p2_word_list=list(set(v1_word_list+v1p1_word_list))

In [155]:
len(v1p2_word_list)

2087

## 训练词频向量

In [156]:
frequency_vector_list = []
tag=0
for word in v1p2_word_list:
    if tag%100==0:
        print(tag,datetime.now())
    pattern=re.compile(word)
    frequency_vector = [len(re.findall(pattern,log))  for log in msg_log_list]
    frequency_vector_list.append(frequency_vector)
    tag+=1

0 2022-04-09 14:53:35.211924
100 2022-04-09 14:53:37.062354
200 2022-04-09 14:53:38.843811
300 2022-04-09 14:53:40.560396
400 2022-04-09 14:53:42.245825
500 2022-04-09 14:53:43.968865
600 2022-04-09 14:53:45.680279
700 2022-04-09 14:53:47.399788
800 2022-04-09 14:53:49.266800
900 2022-04-09 14:53:50.981713
1000 2022-04-09 14:53:52.685844
1100 2022-04-09 14:53:54.396512
1200 2022-04-09 14:53:56.136588
1300 2022-04-09 14:53:57.735333
1400 2022-04-09 14:53:59.471743
1500 2022-04-09 14:54:01.123779
1600 2022-04-09 14:54:02.736899
1700 2022-04-09 14:54:04.386157
1800 2022-04-09 14:54:06.045958
1900 2022-04-09 14:54:07.688432
2000 2022-04-09 14:54:09.431070


In [157]:
frequency_vector_df=pd.DataFrame(frequency_vector_list)
frequency_vector_df=frequency_vector_df.T
frequency_vector_df.columns=v1p2_word_list
new_feature_list=list(msg_log_label_df.columns)[2:-2]
frequency_vector_df[new_feature_list]=msg_log_label_df[new_feature_list]

frequency_vector_df['label']=label_list
frequency_vector_df[['sn','fault_time']]=msg_log_label_df[['sn','fault_time']]
feature=np.array(frequency_vector_df[v1p2_word_list+new_feature_list])

In [158]:
frequency_vector_df

,deasserted drive slot,0x8a,hdd_r_20_status,cpu dimm memory,4560,ps3_status,button buttonmm ev,249,log area reset/cleared,fbb,...,msg_hour_median,msg_hour_mode,msg_minute_max,msg_minute_min,msg_minute_avg,msg_minute_median,msg_minute_mode,label,sn,fault_time
0,0,0,0,0,0,0,0,0,0,0,...,8.0,8,59,0,38.333333,54.0,59,1,SERVER_10001,2020-05-01 10:04:00
1,0,0,0,0,0,0,0,0,0,0,...,9.0,9,48,45,46.444444,46.0,46,2,SERVER_10003,2020-03-28 09:48:00
2,0,0,0,0,0,0,0,0,0,0,...,15.0,15,52,51,51.600000,52.0,52,1,SERVER_10008,2020-02-25 16:12:00
3,0,0,0,0,0,0,0,0,0,0,...,16.0,16,52,46,47.750000,46.5,46,2,SERVER_10008,2020-03-11 18:04:00
4,0,0,0,0,0,0,0,0,0,0,...,16.0,16,7,7,7.000000,7.0,7,3,SERVER_10009,2020-05-08 16:37:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15891,0,0,0,0,0,0,0,0,0,0,...,20.5,19,52,46,49.000000,49.0,46,2,SERVER_9991,2020-08-04 22:49:00
15892,0,0,0,0,0,0,0,0,0,0,...,18.0,18,14,14,14.000000,14.0,14,2,SERVER_9991,2020-10-07 18:42:00
15893,0,0,0,0,0,0,0,0,0,0,...,23.0,23,48,43,45.500000,45.5,43,2,SERVER_9993,2020-05-14 23:50:00
15894,0,0,0,0,0,0,0,0,0,0,...,11.0,11,19,4,11.500000,11.5,4,2,SERVER_9998,2020-05-29 11:25:00


## 训练xgb模型

In [159]:
# xgb模型参数
xgb_params = {
    'booster':'gbtree',
    'objective':'multi:softmax',   # 多分类问题
    'num_class':4,  # 类别数，与multi softmax并用
    'gamma':0.1,    # 用于控制是否后剪枝的参数，越大越保守，一般0.1 0.2的样子
    'max_depth':6,  # 构建树的深度，越大越容易过拟合
    'lambda':2,  # 控制模型复杂度的权重值的L2 正则化项参数，参数越大，模型越不容易过拟合
    'subsample':1, # 随机采样训练样本
    'colsample_bytree':1,# 这个参数默认为1，是每个叶子里面h的和至少是多少
    # 对于正负样本不均衡时的0-1分类而言，假设h在0.01附近，min_child_weight为1
    #意味着叶子节点中最少需要包含100个样本。这个参数非常影响结果，
    # 控制叶子节点中二阶导的和的最小值，该参数值越小，越容易过拟合
    'silent':0,  # 设置成1 则没有运行信息输入，最好是设置成0
    'eta':0.3,  # 如同学习率
    'seed':1000,
    'nthread':16,  #CPU线程数
    #'eval_metric':'auc'
}

# 指标评估
def macro_f1(label,prediction)  -> float:

    """
    计算得分
    :param target_df: [sn,fault_time,label]
    :param submit_df: [sn,fault_time,label]
    :return:
    """

    weights =  [3  /  7,  2  /  7,  1  /  7,  1  /  7]
    macro_F1 =  0.
    for i in  range(len(weights)):
        TP =  np.sum((label==i) & (prediction==i))
        FP =  np.sum((label!= i) & (prediction == i))
        FN =  np.sum((label == i) & (prediction!= i))
        precision = TP /  (TP + FP)  if  (TP + FP)  >  0  else  0
        recall = TP /  (TP + FN)  if  (TP + FN)  >  0  else  0
        F1 =  2  * precision * recall /  (precision + recall)  if  (precision + recall)  >  0  else  0
        macro_F1 += weights[i]  * F1
        
        print('Task %d:\n Prcesion %.2f, Recall %.2f, F1 %.2f' % (i+1, precision, recall, F1))
        
    return macro_F1

In [160]:
random.seed(0)
label=np.array(label_list)
val_mask = [random.random() < 0.3 for _ in range(len(feature))]
train_mask = [not xx for xx in val_mask]
val_feature = feature[val_mask]
val_label = label[val_mask]
train_feature = feature[train_mask]
train_label = label[train_mask]
train_data=xgb.DMatrix(train_feature,label=train_label)
train_feature=xgb.DMatrix(train_feature)
val_feature=xgb.DMatrix(val_feature)

In [161]:
xgb_model=xgb.train(xgb_params,train_data,num_boost_round=500)

[14:54:19] WARNING: ../src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:54:20] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [162]:
train_pred=xgb_model.predict(train_feature)
val_pred=xgb_model.predict(val_feature)

In [163]:
macro_f1(train_label,train_pred)

Task 1:
 Prcesion 1.00, Recall 1.00, F1 1.00
Task 2:
 Prcesion 1.00, Recall 1.00, F1 1.00
Task 3:
 Prcesion 1.00, Recall 1.00, F1 1.00
Task 4:
 Prcesion 1.00, Recall 1.00, F1 1.00


0.9999999999999998

In [164]:
macro_f1(val_label,val_pred)
# 0.6566607725096727
# 0.6640035152684288


Task 1:
 Prcesion 0.54, Recall 0.34, F1 0.42
Task 2:
 Prcesion 0.73, Recall 0.80, F1 0.76
Task 3:
 Prcesion 0.94, Recall 0.96, F1 0.95
Task 4:
 Prcesion 0.89, Recall 0.92, F1 0.91


0.6624843433679657

## 保存验证集，sn+fault_time+日志+标签+重要性前50特征
## 保存验证集类别0_1的验证集，sn+fault_time+日志+标签+重要性前50特征

In [165]:
total_gain_dict=xgb_model.get_score(importance_type='total_gain')
total_gain_dict=dict(sorted(total_gain_dict.items(),key=lambda item:item[1],reverse=True))
feature_names=list(frequency_vector_df.columns)
feature_importance_top_50_dict={}
i=0
for key in total_gain_dict:
    if i<50:
        feature_name=feature_names[int(key[1:])]
        feature_importance_top_50_dict[feature_name]=total_gain_dict[key]
        i+=1
    else:
        break
feature_importance_top_50_dict

{'memory': 14913.8125871041,
 'or': 4281.178824664899,
 'cpu': 3901.2376306080027,
 'processor': 3015.510778684802,
 'server_model': 1086.1872830424006,
 'fault_minute': 1044.8071119966012,
 'msg_minute_avg': 815.5502560702985,
 'msg_cnt': 808.6269430659999,
 'configuration error': 735.4405111364997,
 'caterr': 728.7450824317997,
 'msg_minute_max': 655.4767976624998,
 'msg_minute_min': 648.3979468800993,
 'msg_hour_avg': 607.9135703559996,
 'fault_hour': 607.1334574022001,
 'e': 580.6079890001004,
 'correctable ecc': 491.33322122490034,
 'msg_minute_median': 480.49720456219956,
 'msg_hour_min': 466.57547957839023,
 'msg_minute_mode': 442.3104704041002,
 'msg_hour_max': 440.5003619096998,
 'c': 436.55439594249987,
 'deassert': 417.8479157248006,
 'mcerr': 360.9076179789999,
 'ec': 283.4123014756999,
 'uncorrectable ecc': 242.25308266479982,
 'config': 239.17179976689997,
 '0': 233.52732580850008,
 '0x3b': 230.98126717099998,
 'microcontroller/coprocessor': 190.91058216840003,
 'system':

In [166]:
validation_df = frequency_vector_df[['sn','fault_time']][val_mask]
validation_df['msg_log'] = msg_log_label_df['msg_log'][val_mask]
validation_df['label'] = val_label
validation_df['prediction'] = val_pred
validation_df[list(feature_importance_top_50_dict.keys())] = frequency_vector_df[list(feature_importance_top_50_dict.keys())]
validation_df.to_csv('./pre_contest/v1p7/validation_df.csv',sep=',',index=None)
_0_1_validation_df = validation_df[validation_df['label'].isin([0,1])]
_0_1_validation_df.to_csv('./pre_contest/v1p7/_0_1_validation_df.csv',sep=',',index=None)

In [167]:
_0_1_validation_df

,sn,fault_time,msg_log,label,prediction,memory,or,cpu,processor,server_model,...,1,power supply,0x19,state,cpu1,status,msg_hour_median,in,msg_3_cnt,dimm
40,SERVER_10082,2020-08-12 20:09:00,processor cpu1_status | ierr | deasserted. sy...,1,1.0,0,2,1,1,35,...,1,0,0,2,1,3,19.0,3,4,0
43,SERVER_10087,2020-01-23 04:51:00,processor cpu0_status | ierr | asserted. proc...,1,1.0,0,2,2,2,35,...,1,0,0,0,1,2,3.0,0,2,0
48,SERVER_10094,2020-04-02 10:22:00,processor cpu1_status | ierr | asserted. proc...,1,1.0,0,9,4,4,35,...,2,0,0,2,2,6,9.0,3,9,0
152,SERVER_102,2020-03-10 02:27:00,processor cpu2 status | | asserted. processo...,1,1.0,0,6,4,4,6,...,0,0,0,0,0,4,1.0,3,5,0
223,SERVER_10266,2020-11-14 10:55:00,microcontroller bmc_boot_up | device enabled ...,1,1.0,0,9,6,6,35,...,4,2,0,2,3,10,10.0,3,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15649,SERVER_9726,2020-09-30 19:00:00,processor cpu0_status | ierr | asserted. proc...,1,1.0,0,5,4,4,35,...,2,0,0,2,2,6,18.0,5,8,0
15654,SERVER_9729,2020-08-24 18:18:00,processor cpu0_status | ierr | asserted. proc...,1,1.0,0,9,4,4,35,...,2,0,0,2,2,6,17.0,5,10,0
15658,SERVER_9735,2020-09-20 16:06:00,system acpi power state acpi_pwr_status | s4/...,1,1.0,0,5,2,2,35,...,2,0,0,2,2,4,10.0,3,5,0
15664,SERVER_9747,2020-06-12 04:04:00,processor cpu_caterr | state asserted | asser...,1,1.0,0,2,1,1,49,...,0,0,0,3,0,2,3.0,1,3,0


# 用所有数据进行训练、保存模型、训练集

In [159]:
all_data=xgb.DMatrix(feature,label)
xgb_model_v1p6=xgb.train(xgb_params,all_data,num_boost_round=500)

[19:10:06] WARNING: ../src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:10:07] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [177]:
frequency_vector_df.head()

,bp1_hdd23_status,15,rear2_3_status,security,252,deasserted system firmware error bios poststatus,drive,threshold,outofrange,0x19,...,msg_hour_median,msg_hour_mode,msg_minute_max,msg_minute_min,msg_minute_avg,msg_minute_median,msg_minute_mode,label,sn,fault_time
0,0,0,0,0,0,0,0,0,0,0,...,8.0,8,59,0,38.333333,54.0,59,1,SERVER_10001,2020-05-01 10:04:00
1,0,0,0,0,0,0,0,0,0,0,...,9.0,9,48,45,46.444444,46.0,46,2,SERVER_10003,2020-03-28 09:48:00
2,0,0,0,0,0,0,0,0,0,0,...,15.0,15,52,51,51.600000,52.0,52,1,SERVER_10008,2020-02-25 16:12:00
3,0,0,0,0,0,0,0,0,0,0,...,16.0,16,47,46,46.333333,46.0,46,2,SERVER_10008,2020-03-11 18:04:00
4,0,0,0,0,0,0,8,0,0,0,...,16.0,16,7,7,7.000000,7.0,7,3,SERVER_10009,2020-05-08 16:37:00


In [161]:
frequency_vector_df.to_csv('./pre_contest/v1p6/frequency_vector_df.csv',sep=',',index=None)
file = open('./pre_contest/v1p6/xgb_model_v1p6.model','wb')
pickle.dump(xgb_model_v1p6, file)

## 查看特征重要性、保存特征重要性前300的特征

In [162]:
total_gain_dict=xgb_model_v1p6.get_score(importance_type='total_gain')
total_gain_dict=dict(sorted(total_gain_dict.items(),key=lambda item:item[1],reverse=True))
feature_names=list(frequency_vector_df.columns)
feature_importance_top_300_dict={}
i=0
for key in total_gain_dict:
    if i<300:
        feature_name=feature_names[int(key[1:])]
        feature_importance_top_300_dict[feature_name]=total_gain_dict[key]
        i+=1
    else:
        break
feature_importance_top_300_dict

{'memory': 22207.870354995295,
 'or': 6173.164130015699,
 'cpu': 5820.829303034005,
 'processor': 3735.560395604702,
 'config': 1348.8281070910007,
 'fault_minute': 1267.0539518629002,
 'server_model': 1199.7780455850998,
 'caterr': 1085.3047328632993,
 'msg_minute_avg': 957.5440701405026,
 'msg_cnt': 899.730107435098,
 'msg_minute_max': 855.8012383971012,
 'msg_minute_min': 738.193022992599,
 'msg_hour_avg': 713.2222798615979,
 'fault_hour': 690.9168246005988,
 'e': 644.0394151110005,
 'correctable ecc': 603.6458398336002,
 'msg_minute_mode': 584.3420134322008,
 'c': 534.3769730587993,
 'msg_minute_median': 532.8831313272003,
 'uncorrectable ecc': 521.9578034249002,
 'deassert': 517.4673607651996,
 'msg_hour_max': 461.5146673354003,
 'msg_hour_min': 443.56284308240043,
 'microcontroller/coprocessor': 366.7927114856999,
 'ec': 341.4673784176002,
 '0x3b': 319.4135091712001,
 '0': 258.62340454699995,
 'device disabled': 243.7474478899,
 'on': 234.74257847250013,
 'asserted': 226.00724908

In [101]:
feature_importance_top_300_df=frequency_vector_df[['sn','day','label']+list(feature_importance_top_300_dict.keys())]

In [102]:
# feature_importance_top_300_df.to_csv('./pre_contest/v1p5/feature_importance_top_300_df.csv',sep=',',index=None)

# 对初赛测试集进行预测

## 读取测试集数据、拼接数据

In [178]:
# 读取选手提交数据
submit_a=pd.read_csv('./pre_contest/dataset/preliminary_submit_dataset_a.csv')
submit_log=pd.read_csv('./pre_contest/dataset/preliminary_sel_log_dataset_a.csv')

In [209]:
submit_a

,sn,time,error_type,log_time,msg,msg_len,server_model
0,000d33b21436,2020-09-02 16:42:54,-1,2020-09-02 16:42:54,,0,SM40
1,005c5a9218ba,2020-06-28 19:05:16,-1,2020-06-28 19:05:16,,0,SM99
2,0079283bde6e,2020-04-26 21:32:44,-1,2020-04-26 21:32:44,,0,SM14
3,007bdf23b62f,2020-06-16 18:40:39,-1,2020-06-16 18:40:39,,0,SM93
4,00a577a8e54f,2020-04-07 07:16:55,-1,2020-04-07 07:16:55,,0,SM13
...,...,...,...,...,...,...,...
3006,ffbf46b4af21,2019-12-28 20:10:01,-1,2019-12-28 20:10:01,,0,SM35
3007,ffc229b6cd9a,2020-06-27 02:39:08,-1,2020-06-27 02:39:08,,0,SM49
3008,ffd44698a52b,2020-01-21 15:46:56,-1,2020-01-21 15:46:56,,0,SM66
3009,fff73a9e5bd5,2020-03-01 22:43:43,-1,2020-03-01 22:43:43,,0,SM92


In [211]:
submit_a.groupby(['sn','time']).size()

sn            time               
000d33b21436  2020-09-02 16:42:54    1
005c5a9218ba  2020-06-28 19:05:16    1
0079283bde6e  2020-04-26 21:32:44    1
007bdf23b62f  2020-06-16 18:40:39    1
00a577a8e54f  2020-04-07 07:16:55    1
                                    ..
ffbf46b4af21  2019-12-28 20:10:01    1
ffc229b6cd9a  2020-06-27 02:39:08    1
ffd44698a52b  2020-01-21 15:46:56    1
fff73a9e5bd5  2020-03-01 22:43:43    1
fffd22fffe19  2020-01-21 19:22:56    1
Length: 3011, dtype: int64

In [180]:
submit_log

,sn,time,msg,server_model
0,000d33b21436,2020-09-02 11:38:40,System Boot Initiated BIOS_Boot_Up | Initiate...,SM40
1,000d33b21436,2020-09-02 15:46:23,System Boot Initiated BIOS_Boot_Up | Initiate...,SM40
2,005c5a9218ba,2020-06-28 18:26:25,Memory Memory_Status | Correctable ECC | Asse...,SM99
3,005c5a9218ba,2020-06-28 18:40:26,System ACPI Power State #0x7d | S0/G0: workin...,SM99
4,005c5a9218ba,2020-06-28 18:26:26,Memory Memory_Status | Correctable ECC | Asse...,SM99
...,...,...,...,...
10986,fffd22fffe19,2020-01-21 19:16:01,Microcontroller/Coprocessor #0x16 | Transitio...,SM16
10987,fffd22fffe19,2020-01-21 19:17:03,System Event #0x10 | Timestamp Clock Sync | A...,SM16
10988,fffd22fffe19,2020-01-21 18:32:59,Memory #0xf9 | Uncorrectable ECC | Asserted,SM16
10989,fffd22fffe19,2020-01-21 19:18:14,System Boot Initiated BIOS_Boot_Up | Initiate...,SM16


In [188]:
len(set(submit_a['sn'].drop_duplicates()).intersection(set(submit_log['sn'].drop_duplicates())))

2883

In [291]:
# 读取提交日志数据
submit_log = pd.read_csv('./pre_contest/dataset/preliminary_sel_log_dataset_a.csv')

submit_log['error_type'] = ''
submit_log['log_time'] = pd.to_datetime(submit_log['time'])
submit_log['msg_len'] = submit_log['msg'].apply(lambda d: len(d.split('|')))

submit_log_server_mapping = submit_log[['sn','server_model']].drop_duplicates()


# 读取提交标签数据
submit_a = pd.read_csv('./pre_contest/dataset/preliminary_submit_dataset_a.csv')

submit_a['error_type'] = -1
submit_a['log_time'] = pd.to_datetime(submit_a['fault_time'])
submit_a['msg'] = ''
submit_a['msg_len'] = 0
submit_a['server_model'] = submit_a['sn'].map(dict(zip(submit_log_server_mapping['sn'], 
                                                       submit_log_server_mapping['server_model'])))
submit_a.columns = ['sn','time','error_type','log_time','msg','msg_len','server_model']


# 合并日志表与标签表
submit_combination_df = pd.concat([submit_log, submit_a])
submit_combination_df = submit_combination_df.sort_values(['log_time'], ascending=True)
submit_combination_df = submit_combination_df.reset_index(drop=True)


# 按sn分组，按日志（含标签）时间顺序排列，以报错划分日志，取此次报错前的日志合并

submit_grp_store = []
submit_miss_log_store = []
submit_miss_label_store = []
cnt_miss_label = 0
cnt_with_label = 0
for name, grp in submit_combination_df.groupby(['sn']):
    if len(grp[grp['error_type'] == -1]) == 0:
        cnt_miss_label += 1
        submit_miss_log_store.append(grp)
    elif len(grp[grp['error_type'] == -1]) == 1:
        submit_grp_store.append(grp)
    else:
        cutoff_idx = [-1] + grp.loc[grp['error_type'] == -1].index.tolist() + [grp.index.tolist()[-1]+1]
        for kth in range(len(cutoff_idx)-1): 
            temp_grp = grp.loc[(grp.index <= cutoff_idx[kth+1]) & (grp.index > cutoff_idx[kth])]
            if len(temp_grp) > 0:
                if len(temp_grp[temp_grp['error_type'] == -1]) == 0:
                    cnt_miss_label += 1
                    submit_miss_label_store.append(temp_grp)
                else:
                    cnt_with_label += 1 
                    submit_grp_store.append(temp_grp)
                
                
# 构建新日志-标签表
submit_log_df = pd.DataFrame(columns = ['sn','fault_time','time','msg','server_model']) 

for grp in submit_grp_store:
    fault_time = grp[grp['error_type'] == -1]['time'].iloc[0]
    grp['fault_time'] = fault_time
    if len(grp) > 1:
        temp = grp[grp['error_type'] == ''][['sn','fault_time','time','msg','server_model']]
        submit_log_df = pd.concat([submit_log_df,temp])
    else:
        submit_log_df = pd.concat([submit_log_df,grp])


In [292]:
len(submit_grp_store),len(submit_miss_log_store),len(submit_miss_label_store)

(3011, 0, 1)

## 计算统计特征

In [293]:
submit_log_df['msg_hour'] = submit_log_df['time'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").hour)
submit_log_df['msg_minute'] = submit_log_df['time'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").minute)
submit_log_df['fault_hour'] = submit_log_df['fault_time'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").hour)
submit_log_df['fault_minute'] = submit_log_df['fault_time'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").minute)
# 拼接日志
# 新增 fault_minute，按竖线统计任务数，日志小时和分钟的最大值、最小值、均值、中位数、众数
fault_minute_list = []
msg_1_cnt_list=[]
msg_2_cnt_list=[]
msg_3_cnt_list=[]
msg_4_cnt_list=[]
msg_hour_max_list=[]
msg_hour_min_list=[]
msg_hour_avg_list=[]
msg_hour_median_list=[]
msg_hour_mode_list=[]
msg_minute_max_list=[]
msg_minute_min_list=[]
msg_minute_avg_list=[]
msg_minute_median_list=[]
msg_minute_mode_list=[]


sn_list=[]
fault_time_list=[]
server_model_list=[]
msg_log_list=[]
msg_cnt_list=[]
fault_hour_list=[]
label_list=[]

for msg_log_df in submit_log_label_df.groupby(['sn','fault_time']):
    msg_log_str = ''
    msg_1_cnt = 0
    msg_2_cnt = 0
    msg_3_cnt = 0
    msg_4_cnt = 0
    for info in msg_log_df[1]['msg'].drop_duplicates():
        if info == info:
            msg_log_str=msg_log_str+info.lower()+'.'
            if len(info.split('|')) == 1:
                msg_1_cnt += 1
            elif len(info.split('|')) == 2:
                msg_2_cnt += 1
            elif len(info.split('|')) == 3:
                msg_3_cnt += 1
            else:
                msg_4_cnt += 1
    sm=int(msg_log_df[1].iloc[0]['server_model'][2:])
    
    sn_list.append(msg_log_df[0][0])
    fault_time_list.append(msg_log_df[0][1])
    server_model_list.append(sm)
    msg_log_list.append(msg_log_str)
    msg_cnt_list.append(len(msg_log_df[1]))
    label_list.append(label)
    
    fault_hour_list.append(msg_log_df[1].iloc[0]['fault_hour'])
    fault_minute_list.append(msg_log_df[1].iloc[0]['fault_minute'])
    
    msg_1_cnt_list.append(msg_1_cnt)
    msg_2_cnt_list.append(msg_2_cnt)
    msg_3_cnt_list.append(msg_3_cnt)
    msg_4_cnt_list.append(msg_4_cnt)
    
    msg_hour_max_list.append(msg_log_df[1]['msg_hour'].max())
    msg_hour_min_list.append(msg_log_df[1]['msg_hour'].min())
    msg_hour_avg_list.append(msg_log_df[1]['msg_hour'].mean())
    msg_hour_median_list.append(msg_log_df[1]['msg_hour'].median())
    msg_hour_mode_list.append(msg_log_df[1]['msg_hour'].mode()[0])
    
    msg_minute_max_list.append(msg_log_df[1]['msg_minute'].max())
    msg_minute_min_list.append(msg_log_df[1]['msg_minute'].min())
    msg_minute_avg_list.append(msg_log_df[1]['msg_minute'].mean())
    msg_minute_median_list.append(msg_log_df[1]['msg_minute'].median())
    msg_minute_mode_list.append(msg_log_df[1]['msg_minute'].mode()[0])
    
submit_msg_log_df=pd.DataFrame(
    {
    'sn':sn_list,
    'fault_time':fault_time_list,
    'server_model':server_model_list,
    'msg_cnt':msg_cnt_list,
    'fault_hour':fault_hour_list,
    'fault_minute':fault_minute_list,
    'msg_1_cnt':msg_1_cnt_list,
    'msg_2_cnt':msg_2_cnt_list,
    'msg_3_cnt':msg_3_cnt_list,
    'msg_4_cnt':msg_4_cnt_list,
    'msg_hour_max':msg_hour_max_list,
    'msg_hour_min':msg_hour_min_list,
    'msg_hour_avg':msg_hour_avg_list,
    'msg_hour_median':msg_hour_median_list,
    'msg_hour_mode':msg_hour_mode_list,
    'msg_minute_max':msg_minute_max_list,
    'msg_minute_min':msg_minute_min_list,
    'msg_minute_avg':msg_minute_avg_list,
    'msg_minute_median':msg_minute_median_list,
    'msg_minute_mode':msg_minute_mode_list,
    'msg_log':msg_log_list
    })

In [294]:
submit_msg_log_df

,sn,fault_time,server_model,msg_cnt,fault_hour,fault_minute,msg_1_cnt,msg_2_cnt,msg_3_cnt,msg_4_cnt,...,msg_hour_min,msg_hour_avg,msg_hour_median,msg_hour_mode,msg_minute_max,msg_minute_min,msg_minute_avg,msg_minute_median,msg_minute_mode,msg_log
0,000d33b21436,2020-09-02 16:42:54,40,2,16,42,0,0,2,0,...,11,13.000000,13.0,11,46,38,42.000000,42.0,38,system boot initiated bios_boot_up | initiate...
1,005c5a9218ba,2020-06-28 19:05:16,99,10,19,5,0,0,4,0,...,18,18.000000,18.0,18,40,26,31.400000,26.0,26,memory memory_status | correctable ecc | asse...
2,0079283bde6e,2020-04-26 21:32:44,14,1,21,32,0,0,1,0,...,20,20.000000,20.0,20,54,54,54.000000,54.0,54,power supply psu1_supply | failure detected |...
3,007bdf23b62f,2020-06-16 18:40:39,93,19,18,40,0,0,5,0,...,17,17.000000,17.0,17,45,4,24.421053,16.0,16,memory #0xe2 | correctable ecc | asserted. un...
4,00a577a8e54f,2020-04-07 07:16:55,13,6,7,16,0,0,6,0,...,6,6.000000,6.0,6,52,43,45.666667,45.0,43,memory mem_chg1_status | correctable ecc | as...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3006,ffbf46b4af21,2019-12-28 20:10:01,35,3,20,10,0,0,1,0,...,20,20.000000,20.0,20,9,8,8.666667,9.0,9,memory cpu0c0_dimm_stat | correctable ecc | a...
3007,ffc229b6cd9a,2020-06-27 02:39:08,49,2,2,39,0,0,1,0,...,2,2.000000,2.0,2,36,29,32.500000,32.5,29,memory #0xe2 | correctable ecc | asserted.
3008,ffd44698a52b,2020-01-21 15:46:56,66,2,15,46,0,0,2,0,...,13,13.000000,13.0,13,57,57,57.000000,57.0,57,unknown chassis_control | | asserted. system...
3009,fff73a9e5bd5,2020-03-01 22:43:43,92,6,22,43,0,0,1,0,...,13,13.000000,13.0,13,50,18,44.333333,49.5,50,memory #0x87 | correctable ecc | asserted.


## 计算词频特征向量

In [289]:
# 统计词频向量
submit_frequency_vector_list = []
tag=0
for word in v1p2_word_list:
    if tag%100==0:
        print(tag,datetime.datetime.now())
    pattern=re.compile(word)
    frequency_vector = [len(re.findall(pattern,log))  for log in msg_log_list]
    submit_frequency_vector_list.append(frequency_vector)
    tag+=1                                                              

0 2022-04-04 23:20:59.972675
100 2022-04-04 23:21:00.396766
200 2022-04-04 23:21:00.792601
300 2022-04-04 23:21:01.159176
400 2022-04-04 23:21:01.497585
500 2022-04-04 23:21:01.885660
600 2022-04-04 23:21:02.235019
700 2022-04-04 23:21:02.579013
800 2022-04-04 23:21:03.006614
900 2022-04-04 23:21:03.306641
1000 2022-04-04 23:21:03.616905
1100 2022-04-04 23:21:03.983513
1200 2022-04-04 23:21:04.293546
1300 2022-04-04 23:21:04.651483
1400 2022-04-04 23:21:04.995835
1500 2022-04-04 23:21:05.326449
1600 2022-04-04 23:21:05.617902
1700 2022-04-04 23:21:05.939220
1800 2022-04-04 23:21:06.260225
1900 2022-04-04 23:21:06.578668
2000 2022-04-04 23:21:06.903447


In [295]:
submit_frequency_vector_df=pd.DataFrame(submit_frequency_vector_list)
submit_frequency_vector_df=submit_frequency_vector_df.T
submit_frequency_vector_df.columns=v1p2_word_list
submit_frequency_vector_df[new_feature_list]=submit_msg_log_df[new_feature_list]
submit_frequency_vector_df[['sn','fault_time']]=submit_msg_log_df[['sn','fault_time']]
feature=np.array(submit_frequency_vector_df[v1p2_word_list+new_feature_list])  

# submit_frequency_vector_df.to_csv('./pre_contest/v1p6/submit_frequency_vector_df.csv',sep=',',index=None)

In [296]:
submit_frequency_vector_df

,bp1_hdd23_status,15,rear2_3_status,security,252,deasserted system firmware error bios poststatus,drive,threshold,outofrange,0x19,...,msg_hour_avg,msg_hour_median,msg_hour_mode,msg_minute_max,msg_minute_min,msg_minute_avg,msg_minute_median,msg_minute_mode,sn,fault_time
0,0,0,0,0,0,0,0,0,0,0,...,13.000000,13.0,11,46,38,42.000000,42.0,38,000d33b21436,2020-09-02 16:42:54
1,0,0,0,0,0,0,0,0,0,0,...,18.000000,18.0,18,40,26,31.400000,26.0,26,005c5a9218ba,2020-06-28 19:05:16
2,0,0,0,0,0,0,0,0,0,0,...,20.000000,20.0,20,54,54,54.000000,54.0,54,0079283bde6e,2020-04-26 21:32:44
3,0,0,0,0,0,0,0,0,0,0,...,17.000000,17.0,17,45,4,24.421053,16.0,16,007bdf23b62f,2020-06-16 18:40:39
4,0,0,0,0,0,0,0,0,0,0,...,6.000000,6.0,6,52,43,45.666667,45.0,43,00a577a8e54f,2020-04-07 07:16:55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3006,0,0,0,0,0,0,0,0,0,0,...,20.000000,20.0,20,9,8,8.666667,9.0,9,ffbf46b4af21,2019-12-28 20:10:01
3007,0,0,0,0,0,0,0,0,0,0,...,2.000000,2.0,2,36,29,32.500000,32.5,29,ffc229b6cd9a,2020-06-27 02:39:08
3008,0,0,0,0,0,0,0,0,0,0,...,13.000000,13.0,13,57,57,57.000000,57.0,57,ffd44698a52b,2020-01-21 15:46:56
3009,0,0,0,0,0,0,0,0,0,0,...,13.000000,13.0,13,50,18,44.333333,49.5,50,fff73a9e5bd5,2020-03-01 22:43:43


## 对测试集进行预测，保存特征和预测结果

In [298]:
test_feature=xgb.DMatrix(feature)
submit_frequency_vector_df['prediction']=xgb_model_v1p6.predict(test_feature)
preliminary_submit_dataset_a=submit_frequency_vector_df[['sn','fault_time','prediction']]
preliminary_submit_dataset_a['label']=preliminary_submit_dataset_a['prediction'].apply(lambda x : int(x))
preliminary_submit_dataset_a=preliminary_submit_dataset_a[['sn','fault_time','label']]

/var/folders/bh/sv71wm991pb8sm02bw8rmrfc0000gp/T/ipykernel_34755/616074248.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preliminary_submit_dataset_a['label']=preliminary_submit_dataset_a['prediction'].apply(lambda x : int(x))


In [299]:
preliminary_submit_dataset_a.to_csv('pre_contest/v1p6/preliminary_submit_dataset_a_v1p6.csv',sep=',',index=None)